# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [105]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [106]:
# Reaing the csv file
cities_csv = "../Weatherpy/output_data/weather_city_data.csv"
weather_cities = pd.read_csv(cities_csv)
weather_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cape town,-33.9258,18.4232,57.99,82,0,8.05,za,1619736414
1,isangel,-19.5500,169.2667,77.00,78,75,6.91,vu,1619736705
2,dhidhdhoo,6.8833,73.1000,83.23,74,100,6.22,mv,1619736654
3,ushuaia,-54.8000,-68.3000,34.88,87,100,4.45,ar,1619736706
4,iqaluit,63.7506,-68.5145,32.00,100,90,4.61,ca,1619736706
...,...,...,...,...,...,...,...,...,...
565,asenovgrad,42.0167,24.8667,57.20,82,35,8.05,bg,1619736930
566,bonavista,48.6499,-53.1147,44.60,76,86,3.44,ca,1619736930
567,hasaki,35.7333,140.8333,64.40,59,20,6.91,jp,1619736931
568,cao bang,22.6667,106.2500,66.34,87,93,0.74,vn,1619736931


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [107]:
# Configuring the gmaps
gmaps.configure(api_key=g_key)

In [108]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_cities[["Lat","Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_cities["Humidity"].astype(float)

In [109]:
locations

,Lat,Lng
0,-33.9258,18.4232
1,-19.5500,169.2667
2,6.8833,73.1000
3,-54.8000,-68.3000
4,63.7506,-68.5145
...,...,...
565,42.0167,24.8667
566,48.6499,-53.1147
567,35.7333,140.8333
568,22.6667,106.2500


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [110]:
# DataFrame to find ideal weather condition
# A max temperature lower than 80 degrees but higher than 70.

ideal_weather = weather_cities.loc[
    (weather_cities["Max Temp"] >=70) & 
    (weather_cities["Max Temp"] <=80)&
    (weather_cities["Wind Speed"] <=10)&
    (weather_cities["Cloudiness"] == 0)]

len(ideal_weather)


# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
ideal_weather= ideal_weather.dropna()

In [111]:
ideal_weather.reset_index()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,116,turayf,31.6725,38.6637,71.60,28,0,9.22,sa,1619736751
1,147,khorramshahr,30.4397,48.1664,77.00,41,0,4.61,ir,1619736661
2,186,cayenne,4.9333,-52.3333,78.01,94,0,2.30,gf,1619736591
3,374,buraydah,26.3260,43.9750,77.00,22,0,8.05,sa,1619736729
4,413,sakakah,29.9697,40.2064,71.60,16,0,4.61,sa,1619736869
5,440,inhambane,-23.8650,35.3833,73.51,78,0,8.46,mz,1619736879
6,481,riyadh,24.6877,46.7219,75.20,35,0,5.10,sa,1619736692


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [112]:
# Store into variable named hotel_df.
hotel_df = ideal_weather.reset_index()

In [113]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [114]:
# Hit the Google Places API for each city's coordinates.

# set up a parameters dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

curr_hotel
# Looping through all the information retrieving
for index,row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")



In [115]:
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,116,turayf,31.6725,38.6637,71.60,28,0,9.22,sa,1619736751,Swiss Spirit Hotel & Suites Turaif
1,147,khorramshahr,30.4397,48.1664,77.00,41,0,4.61,ir,1619736661,دفتر نمایندگی شرکت بازاریابان ایرانیان زمینBIZ
2,186,cayenne,4.9333,-52.3333,78.01,94,0,2.30,gf,1619736591,Hôtel Le Dronmi
3,374,buraydah,26.3260,43.9750,77.00,22,0,8.05,sa,1619736729,فندق راحة السلام
4,413,sakakah,29.9697,40.2064,71.60,16,0,4.61,sa,1619736869,Fakhamat Al Orjoana Apartment
5,440,inhambane,-23.8650,35.3833,73.51,78,0,8.46,mz,1619736879,Oceano
6,481,riyadh,24.6877,46.7219,75.20,35,0,5.10,sa,1619736692,"InterContinental Riyadh, an IHG Hotel"


In [116]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [104]:
# Add marker layer ontop of heat map
# Set parameters to search for hotels with 5000 meters.
locations.head()
# dataframe with columns ('latitude', 'longitude', 'magnitude')
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heatmap_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [102]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 5
